<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500 height=450/></p>

<h3 style="text-align: center;"><b>"Глубокое обучение". Продвинутый поток</b></h3>

<h2 style="text-align: center;"><b>Семинар 3. Задачи по Numpy</b></h2>

Вам предлагается решить задачи ниже.

In [0]:
import numpy as np

### Верхнетреугольная матрица
Напишите код, чтобы получить верхнетреугольную матрицу, обнулив все ненужные элементы.
(Верхнетреугольная матрица - такая матрица, что все числа под главной диагональю = 0)

Самое простое решение - загуглить, потому что почти для всего в numpy есть готовое решение:)

In [0]:
def get_triangular_matrix(A):
    return np.triu(A)

In [4]:
A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

get_triangular_matrix(A)

array([[1, 2, 3],
       [0, 5, 6],
       [0, 0, 9]])

# Broadcasting

На самом деле, можно поэлементно складывать/умножать итд массивы с несовпадающими размерами. Это называется broadcasting (ссылка на документацию https://docs.scipy.org/doc/numpy-1.15.0/user/basics.broadcasting.html). В процессе broadcasting'a numpy  пытается сделать shape обоих массивов одинаковым, чтобы провести нужную операцию.

Для этого проводится сравнение размерностей, начиная с последних размерностей. Размерности совместимы, если
1. Количество элементов по этой размерности совпадает.
2. Количество элементов в одной из размернойстей равно 1.

Допустим, у нас было два массива, которые мы складываем:

$shape_1 = (1, 2, 1)$,
$shape_2 = (2, 100)$.

Тогда можно считать, что после броадкастинга получатся массивы с shape = (1, 2, 100) и уже они сложатся

In [0]:
a = np.array([1, 2, 3])
b = np.array([
    [0, 1, 2],
    [3, 4, 5]
])

print(a.shape)
print(b.shape)
print((a + b).shape)

In [0]:
a = a.reshape(1, 1, 3)
print(a.shape)
print(b.shape)
print((a + b).shape)

Вообще говоря, чем использовать Broadcating, лучше в явном виде прописывать все размерности. Если размерности обоих тензоров совпадают (например, они оба трёхмерные), тогда необходимо, чтобы соответствующие элементы кортежей формы либо совпадали, либо один из них был равен 1.

Пример:

In [0]:
a + b[np.newaxis, :, :].shape

### Ромбик
Напишите код, чтобы в квадратной матрице 5 * 5 из нулей получить ромб, который касается середин сторон квадрата

In [0]:
A = np.zeros(shape=(5,5))

a = np.array([2,1,0,1,2])[None,:]
b = np.array([2,1,0,1,2])[:, None]

A = (a+b == 2)*1


# Как правильно выбрать оси?



### Безобидная задача на кумулятивные суммы
Дана матрица $M \times N$. Напишите функцию, которая возвращает вектор средних значений по вертикали. 

In [17]:
def vertical_means(A):
    return A.mean(axis=1)

A = np.array([[1,3],
              [2,4]] )

vertical_means(A).astype(int)

array([2, 3])

**Проблемы:**

* Что такое вертикальная ось? 
* По какой оси необходимо суммировать?
* Как не ошибиться?

**Ответ:** Операции всегда производятся по той оси, которая **исчезнет** после применения операции.

# Батчи
Как вы, возможно, знаете, обучение нейронных сетей происходит последовательной подачей на вход нейронной сети объектов из обучающей выборки. Представим, например, что объекты --- это картинки в формате RGB.

Чтобы нейронная сеть обучалась быстрее, объекты в неё подаются не по одному, а **батчами** из N объектов. Итак, на вход нейронной сети подаётся четырёхмерный (!!!) тензор формы (batch_size, num_channels, height, width). Наверное, вы уже убедились, что знать, где правильно ставить оси, просто необходимо.

Благо, есть простое решение: можно просто НЕ ТРОГАТЬ нулевую ось.

### Стандартизация картинки

На диске лежит файл image_batch.npy (здесь в ноутбуке мы просто генеируем батч с помощью функции np.random.randint). В нем лежит батч (массив) картинок в формате numpy. Каждая картинка задана как трехмерная матрица формата (num_channels, width, height). Нужно стандартизировать каналы одного пикселя, т. е. сделать так, чтобы для каждого пикселя среднее по всем каналам стало равно 0, а стандартное отклонение 1.

In [0]:
def normalize_pictures(A):
    """
    param A: np.array[batch_size, num_channels, width, height]
    """
    m = np.mean(A, axis=(2,3))
    sigma = np.std(A, axis=(2,3))
    ret = (A-m[:,:, np.newaxis, np.newaxis])/sigma[:,:,np.newaxis, np.newaxis]
    return ret

In [30]:
batch = np.random.randint(0, 256, (1, 1, 10, 10))
B = normalize_pictures(batch)
print(B.mean())
print(B.std())

-1.7319479184152443e-16
1.0


### Стандартизация и транспонирование 
Задание то же самое, но нужно стандартизировать все пиксели внутри каждой картинки и сменить формат (batch_size, num_channels, x_coord, y_coord) на (batch_size, x_coord, y_coord, num_channels). Такой формат обычно удобнее для разных вычислений и вообще выглядит более естественным, но GPU более эффективно работают с первым. 

In [0]:
def normalize_and_transpose_pictures(A):
    """
    param A: np.array[batch_size, num_channels, width, height]
    """
    <YOUR CODE>
    
    return <YOUR CODE>

In [0]:
res_transposed = normalize_and_transpose_pictures(batch) 
res_transposed.mean(axis=(1, 2))

### Сжатие фотографии
Теперь мы хотим снизить качество батча фотографий --- разбить каждое изображение на квадратики 2*2 и усреднить внутри них значения по каждому каналу отдельно.

In [0]:
def low_quality(batch_images):
    """
    <Write parameter shapes>
    """
    
    <YOUR CODE>
    
    return <YOUR CODE>

Протестируем на реальной картинке

In [0]:
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('./rose.jpeg')
print(img.shape)
# в картинке сначала идут оси и только потом каналы + ее стороны не четные
# Избавимся от нечетности
img_padded = np.pad(img, [(0,1), (0, 0), (0, 0)], mode='constant')
print(img_padded.shape)
# Превратим картинку в батч
batch = np.array([img_padded])

# Применим наш код
sum_of_pixels = batch[:, ::2, ::2, :] + batch[:, ::2, 1::2, :] + batch[:, 1::2, ::2, :] + batch[:, 1::2, 1::2, :]
low_res_img = (sum_of_pixels / 4).astype(np.uint8)[0]

plt.imshow(img)
plt.show()
plt.imshow(low_res_img)
plt.show()

### Ридж
Дана квадратная матрица $A$ и массив $b$ соответствующей длины. Прибавьте элементы массива $b$ к главной диагонали матрицы $A$

In [0]:
def upgraded_plus(A, b):
    <YOUR CODE>
    
    return <YOUR CODE>